In [1]:
import math
from lib.func import fetch, reduce

In [47]:
data = '''32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483'''.split("\n")

In [49]:
data = fetch(7).split("\n")

### Functions

In [44]:
def buildPlayer(row):
    # Build player object from row data
    hand, wager = row.split()
    return { "hand": hand, "wager": int(wager) }

def getCardValue(card, joker):
    # Get card value for comparison
    val = { "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9": 9, "T": 10, "J": (11 if not joker else 1) , "Q": 12, "K": 13, "A": 14 }
    return val[card]

def updateCardCount(cards, card):
    # Update card count when sorting for hand value
    if not cards.get(card):
        cards[card] = 0
    cards[card] += 1

def getHandIndex(card_counts):
    # Get hand idx by hand value
    val = card_counts[0]

    # Get values that are appropriate to order
    if len(card_counts) > 1:
        val += 0 if card_counts[1] == 1 else 0.5

    # Convert the order to be integers 0-6 for array idx later
    x = int(val * 2) - 2 if val < 5 else 7
    return x if not x else x - 1

def getCardToAddJokers(cards):
    # Get most populous card to which Joker numbers will be added
    most = 0
    most_card = None

    keys = list(cards)

    for key in keys:
        if cards[key] > most:
            most = cards[key]
            most_card = key

    return most_card

def getSortedCardCounts(hand, joker):
    # Get the index for the given hand after sorting
    cards = {}
    for card in hand:
        updateCardCount(cards, card)
    
    # If joker rules in place
    if joker:
        if cards.get("J"):
            count = cards.pop("J")
            c = getCardToAddJokers(cards)
            if not c:
                c = "J"
                cards[c] = 0
            cards[c] = cards[c] + count

    return sorted(list(cards.values()), reverse=True)

def compareHands(player0, player1, joker):
    # Return 0 if player0 has a hand of higher value, 1 if player1 does
    for i in range(len(player0['hand'])):
        if getCardValue(player0['hand'][i], joker) > getCardValue(player1['hand'][i], joker):
            return 0
        if getCardValue(player0['hand'][i], joker) < getCardValue(player1['hand'][i], joker):
            return 1

def insertPlayerSorted(players, player, joker):
    # Binary search of objects placing higher ranking to the right
    lo, hi = [0, len(players) - 1]

    while lo < hi:
        mid = math.floor((hi - lo) / 2) + lo
        if not compareHands(players[mid], player, joker):
            hi = mid - 1
        else:
            lo = mid + 1

    playerIsHigher = not compareHands(players[lo], player, joker)
    players.insert(lo + (0 if playerIsHigher else 1), player)

def sortPlayerIntoHands(hands, player, joker=False):
    # Sort current player into appropriate hand index
    card_counts = getSortedCardCounts(player['hand'], joker)
    hand_idx = getHandIndex(card_counts)

    if not hands[hand_idx]:
        hands[hand_idx] = [player]
    else:
        insertPlayerSorted(hands[hand_idx], player, joker)

def buildFinalPlayerArray(hands):
    # Flatten array of arrays, ignore empty indexes
    arr = []
    for hand in hands:
        if hand:
            for player in hand:
                arr.append(player)
    return arr


### Solution

In [57]:
def solveDay07(data, part=1):
    hands = [None] * 7
    
    for row in data:
        player = buildPlayer(row)
        sortPlayerIntoHands(hands, player, joker=(part==2))

    arr = buildFinalPlayerArray(hands)

    winnings_arr = [player['wager'] * (i + 1) for i, player in enumerate(arr)]
    return reduce(winnings_arr)
    

In [56]:
solveDay07(data, part=2)

254412181